In [12]:
import pandas as pd
import getpass
import sqlalchemy as sa

In [55]:
driver = 'mysql+pymysql'
user = 'root'
password = getpass.getpass(prompt='Your passwords:'"")
ip = '127.0.0.1'

Your passwords:········


In [56]:
connection_string = f'{driver}://{user}:{password}@{ip}'

In [57]:
db_connection = sa.create_engine(connection_string)

In [58]:
insp = sa.inspect(db_connection)
db_list = insp.get_schema_names()
print(db_list)

['bank', 'information_schema', 'mysql', 'performance_schema', 'sakila']


#### Lab SQL Subqueries

In [61]:
# Retrieve data from MYSQL into Pandas Data Frame
query = "SELECT * FROM sakila.inventory"
sakila_inventory = pd.read_sql_query(query, db_connection)
sakila_inventory.head()

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17


In [62]:
sakila_inventory

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17
...,...,...,...,...
4576,4577,1000,1,2006-02-15 05:09:17
4577,4578,1000,2,2006-02-15 05:09:17
4578,4579,1000,2,2006-02-15 05:09:17
4579,4580,1000,2,2006-02-15 05:09:17


1. How many copies of the film Hunchback Impossible exist in the inventory system?

In [84]:
query1 = '''SELECT count(inventory_id), film_id FROM sakila.inventory
WHERE film_id = (SELECT film_id from sakila.film
WHERE film.title = 'Hunchback Impossible')
GROUP BY film_id'''

In [85]:
copies_film_HI = pd.read_sql_query(query1, db_connection)

In [86]:
copies_film_HI

,count(inventory_id),film_id
0,6,439


2. List all films whose length is longer than the average of all the film

In [88]:
query2 = "SELECT title, length FROM sakila.film WHERE length > (SELECT avg(length) from sakila.film) ORDER BY length desc"

In [89]:
query1
films_longer_than_average = pd.read_sql_query(query2, db_connection)

In [90]:
films_longer_than_average

,title,length
0,CHICAGO NORTH,185
1,CONTROL ANTHEM,185
2,DARN FORRESTER,185
3,GANGS PRIDE,185
4,HOME PITY,185
...,...,...
484,RESURRECTION SILVERADO,117
485,DUCK RACER,116
486,INSTINCT AIRPORT,116
487,MADIGAN DORADO,116


3. Use subqueries to display all actors who appear in the film Alone Trip.

In [103]:
query3 = '''SELECT first_name, last_name FROM sakila.actor
WHERE actor_id IN (SELECT actor_id FROM sakila.film_actor
WHERE film_id = (SELECT film_id FROM sakila.film
WHERE title = "Alone Trip"))'''

In [104]:
actors_AT = pd.read_sql_query(query3, db_connection)

In [105]:
actors_AT

,first_name,last_name
0,ED,CHASE
1,KARL,BERRY
2,UMA,WOOD
3,WOODY,JOLIE
4,SPENCER,DEPP
5,CHRIS,DEPP
6,LAURENCE,BULLOCK
7,RENEE,BALL


4. Sales have been lagging among young families, and you wih to target all family movies for a promotion. Identify all movies categorized as family films.

In [107]:
query4 = '''SELECT title, film_id from sakila.film
WHERE film_id in (SELECT film_id FROM sakila.film_category
WHERE category_id in (SELECT category_id FROM sakila.category
WHERE name = "Family"))'''

In [109]:
family_films = pd.read_sql_query(query4, db_connection)

In [110]:
family_films

,title,film_id
0,AFRICAN EGG,5
1,APACHE DIVINE,31
2,ATLANTIS CAUSE,43
3,BAKED CLEOPATRA,50
4,BANG KWAI,53
...,...,...
64,SPLASH GUMP,832
65,SUNSET RACER,866
66,SUPER WYOMING,867
67,VIRTUAL SPOILERS,946


5. Get name and email from customers from Canada using subqueries. Do the same with joins. 
Note that to create a join, you will have to identify the correct tables
with their primary keys and foreign keys, that will help you get the relevant information.

In [121]:
query5 = '''SELECT first_name, last_name, email FROM sakila.customer
WHERE address_id in (SELECT address_id FROM sakila.address
WHERE city_id in (SELECT city_id FROM sakila.city
WHERE country_id in (SELECT country_id FROM sakila.country
WHERE country = "Canada")))'''

In [122]:
canada_customers = pd.read_sql_query(query5, db_connection)

In [123]:
canada_customers

,first_name,last_name,email
0,DERRICK,BOURQUE,DERRICK.BOURQUE@sakilacustomer.org
1,DARRELL,POWER,DARRELL.POWER@sakilacustomer.org
2,LORETTA,CARPENTER,LORETTA.CARPENTER@sakilacustomer.org
3,CURTIS,IRBY,CURTIS.IRBY@sakilacustomer.org
4,TROY,QUIGLEY,TROY.QUIGLEY@sakilacustomer.org


INNER JOIN:

In [124]:
query6 = '''SELECT first_name, last_name, email FROM sakila.customer
INNER JOIN address
USING (address_id)
INNER JOIN city
USING (city_id)
INNER JOIN country
USING (country_id)
WHERE country = "Canada"'''

In [125]:
canada_customers_i = pd.read_sql_query(query6, db_connection)

OperationalError: (pymysql.err.OperationalError) (1046, 'No database selected')
[SQL: SELECT first_name, last_name, email FROM customer
INNER JOIN address
USING (address_id)
INNER JOIN city
USING (city_id)
INNER JOIN country
USING (country_id)
WHERE country = "Canada"]
(Background on this error at: https://sqlalche.me/e/14/e3q8)